<a href="https://colab.research.google.com/github/bhoomireddyvijayakumari/Food_Waste_Management/blob/main/food_waste_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Title - Food Waste Management

##### **Project Type**    - EDA/SQL/Streamlit
##### **Contribution**    - Individual


# **Project Summary -**

This project involved the comprehensive development of a full-stack Local Food Wastage Management System, executed methodically in a single Jupyter notebook. The process began with environment setup, mounting Google Drive for data access and installing and configuring a local MySQL server, including setting up authentication and creating the dedicated foodwaste_db database. The core of the project was the database schema design, where four relational tables (providers, receivers, food_listings, and claims) were meticulously defined with appropriate primary keys, foreign key constraints, and ENUM types for status fields to ensure data integrity and model real-world relationships between donors, food items, and recipients.

Following the schema creation, the data ingestion phase loaded sample data from four provided CSV files. Robust Python functions were written to read, normalize column names, and handle type conversions—especially for dates—before inserting the records into their respective MySQL tables with upsert logic to avoid duplicates. With the database populated, the project advanced to data analysis, executing 15 predefined SQL queries to generate business intelligence. These queries extracted valuable insights on geographic distribution of providers/receivers, top contributing entities, claim status percentages, expiry trends, and supply-demand gaps across cities. The results of these analyses were automatically exported as a set of CSV reports for further review.

The final and most significant phase was the development of a Streamlit web application. A complete app.py file was written in-line, creating a multi-tab, interactive interface. The application features a public-facing tab for browsing and filtering available food donations with integrated contact links (email, phone, WhatsApp), a secure administrative tab for full CRUD (Create, Read, Update, Delete) operations on all database entities, a dedicated analytics tab displaying the results of all 15 SQL queries, and an insights tab featuring Plotly charts visualizing top providers, high-demand locations, and wastage trends. The notebook concluded by attempting to deploy the application using ngrok to create a public URL, making the locally hosted Streamlit app accessible over the internet, thereby demonstrating a fully functional, end-to-end solution that connects food donors with recipients to effectively reduce waste.

# **GitHub Link -**. https://github.com/bhoomireddyvijayakumari/Food_Waste_Management.git

# **Problem Statement**


Food wastage is a significant issue, with many households and restaurants discarding surplus food while numerous people struggle with food insecurity.


This project aims to develop a Local Food Wastage Management System, where:
Restaurants and individuals can list surplus food.

- NGOs or individuals in need can claim the food.
- SQL stores available food details and locations.
- A Streamlit app enables interaction, filtering, CRUD operation and visualization.


# ***Let's Begin !***

## ***1. Know Your Data***

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import Libraries

In [2]:
import streamlit as st
import pandas as pd
import mysql.connector
import mysql.connector as mysql
from mysql.connector import Error
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, date
import os, subprocess, textwrap, json, sys, getpass, re
from dateutil import parser as dateparser

import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
providers_csv = "/content/drive/My Drive/Colab Notebooks/FWM/providers_data.csv"
receivers_csv = "/content/drive/My Drive/Colab Notebooks/FWM/receivers_data.csv"
food_listings_csv = "/content/drive/My Drive/Colab Notebooks/FWM/food_listings_data.csv"
claims_csv = "/content/drive/My Drive/Colab Notebooks/FWM/claims_data.csv"

In [4]:
#Connecting to mysql server on colab

!apt-get update -q
!apt-get install -y mysql-server


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lis

In [5]:
!service mysql start


 * Starting MySQL database server mysqld
   ...done.


In [6]:
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'; FLUSH PRIVILEGES;"
!mysql -uroot -proot -e "CREATE DATABASE IF NOT EXISTS foodwaste_db;"


ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)
mysql: [Warning] Using a password on the command line interface can be insecure.


In [7]:
#Test Python connection to MySQL

DB_CONFIG = dict(
    host="127.0.0.1",
    user="root",
    password="root",
    database="foodwaste_db",
    auth_plugin="mysql_native_password"
)

con = mysql.connect(**DB_CONFIG)
cur = con.cursor()
cur.execute("SELECT DATABASE();")
print("Connected to:", cur.fetchone()[0])
cur.close()
con.close()


Connected to: foodwaste_db


#Create tables (Schema)

In [8]:
schema_sql = """
DROP TABLE IF EXISTS claims;
DROP TABLE IF EXISTS food_listings;
DROP TABLE IF EXISTS receivers;
DROP TABLE IF EXISTS providers;

CREATE TABLE providers (
  provider_id   INT PRIMARY KEY,
  name          VARCHAR(255) NOT NULL,
  type          VARCHAR(100) NOT NULL,
  address       VARCHAR(255),
  city          VARCHAR(100),
  contact       VARCHAR(100)
);

CREATE TABLE receivers (
  receiver_id   INT PRIMARY KEY,
  name          VARCHAR(255) NOT NULL,
  type          VARCHAR(100) NOT NULL,
  city          VARCHAR(100),
  contact       VARCHAR(100)
);

CREATE TABLE food_listings (
  food_id       INT PRIMARY KEY,
  food_name     VARCHAR(255) NOT NULL,
  quantity      INT NOT NULL,
  expiry_date   DATE NOT NULL,
  provider_id   INT NOT NULL,
  provider_type VARCHAR(100),
  location      VARCHAR(100),
  food_type     VARCHAR(50),
  meal_type     VARCHAR(50),
  status        ENUM('Available','Reserved','Claimed','Expired') DEFAULT 'Available',
  FOREIGN KEY (provider_id) REFERENCES providers(provider_id)
    ON UPDATE CASCADE ON DELETE RESTRICT
);

CREATE TABLE claims (
  claim_id    INT PRIMARY KEY,
  food_id     INT NOT NULL,
  receiver_id INT NOT NULL,
  status      ENUM('Pending','Completed','Cancelled') NOT NULL,
  timestamp   DATETIME NOT NULL,
  FOREIGN KEY (food_id) REFERENCES food_listings(food_id)
    ON UPDATE CASCADE ON DELETE RESTRICT,
  FOREIGN KEY (receiver_id) REFERENCES receivers(receiver_id)
    ON UPDATE CASCADE ON DELETE RESTRICT
);
"""

con = mysql.connect(**DB_CONFIG)
cur = con.cursor()
for stmt in schema_sql.split(";"):
    if stmt.strip():
        cur.execute(stmt)
con.commit()
cur.close()
con.close()
print("Schema created ✅")


Schema created ✅


#Load CSVs into MySQL

In [9]:
def connect_db():
    return mysql.connect(**DB_CONFIG)

def normalize_columns(df):
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    return df

def coerce_date(val):
    if pd.isna(val): return None
    try: return pd.to_datetime(val).date()
    except: return dateparser.parse(str(val)).date()


def load_providers():
    df = normalize_columns(pd.read_csv(providers_csv))
    con = connect_db(); cur = con.cursor()
    for _, r in df.iterrows():
        cur.execute("""
        INSERT INTO providers(provider_id,name,type,address,city,contact)
        VALUES(%s,%s,%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE name=VALUES(name), type=VALUES(type),
                                address=VALUES(address), city=VALUES(city), contact=VALUES(contact)
        """, (int(r['provider_id']), str(r['name']), str(r['type']),
              str(r.get('address','')), str(r.get('city','')), str(r.get('contact',''))))
    con.commit(); cur.close(); con.close()

def load_receivers():
    df = normalize_columns(pd.read_csv(receivers_csv))
    con = connect_db(); cur = con.cursor()
    for _, r in df.iterrows():
        cur.execute("""
        INSERT INTO receivers(receiver_id,name,type,city,contact)
        VALUES(%s,%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE name=VALUES(name), type=VALUES(type),
                                city=VALUES(city), contact=VALUES(contact)
        """, (int(r['receiver_id']), str(r['name']), str(r['type']),
              str(r.get('city','')), str(r.get('contact',''))))
    con.commit(); cur.close(); con.close()

def load_food_listings():
    df = normalize_columns(pd.read_csv(food_listings_csv))
    con = connect_db(); cur = con.cursor()
    for _, r in df.iterrows():
        cur.execute("""
        INSERT INTO food_listings
         (food_id, food_name, quantity, expiry_date, provider_id, provider_type, location, food_type, meal_type, status)
        VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s, COALESCE(%s,'Available'))
        ON DUPLICATE KEY UPDATE food_name=VALUES(food_name), quantity=VALUES(quantity),
            expiry_date=VALUES(expiry_date), provider_id=VALUES(provider_id),
            provider_type=VALUES(provider_type), location=VALUES(location),
            food_type=VALUES(food_type), meal_type=VALUES(meal_type), status=VALUES(status)
        """, (
            int(r['food_id']), str(r['food_name']), int(r['quantity']),
            coerce_date(r['expiry_date']), int(r['provider_id']),
            str(r.get('provider_type','')), str(r.get('location','')),
            str(r.get('food_type','')), str(r.get('meal_type','')), str(r.get('status','Available'))
        ))
    con.commit(); cur.close(); con.close()

def load_claims():
    df = normalize_columns(pd.read_csv(claims_csv))
    con = connect_db(); cur = con.cursor()
    for _, r in df.iterrows():
        ts = pd.to_datetime(r['timestamp']).to_pydatetime()  # ✅ fix
        cur.execute("""
        INSERT INTO claims(claim_id,food_id,receiver_id,status,timestamp)
        VALUES(%s,%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE food_id=VALUES(food_id), receiver_id=VALUES(receiver_id),
                                status=VALUES(status), timestamp=VALUES(timestamp)
        """, (int(r['claim_id']), int(r['food_id']), int(r['receiver_id']),
              str(r['status']), ts))
    con.commit(); cur.close(); con.close()



load_providers()
load_receivers()
load_food_listings()
load_claims()
print("CSV data loaded ✅")

CSV data loaded ✅


#SQL Analysis

In [10]:
import os
import pandas as pd
import mysql.connector as mysql

os.makedirs("/mnt/data/reports", exist_ok=True)

def sql_df(q):
    con = mysql.connect(**DB_CONFIG)
    df = pd.read_sql(q, con)
    con.close()
    return df

reports = {
    # Categories
    "wastage_by_food_type.csv": """
        SELECT food_type,
               SUM(CASE WHEN status='Expired' THEN 1 ELSE 0 END) AS expired_items,
               SUM(CASE WHEN status='Available' THEN 1 ELSE 0 END) AS still_unclaimed,
               COUNT(*) AS total_items
        FROM food_listings
        GROUP BY food_type
        ORDER BY expired_items DESC, total_items DESC;
    """,
    # Locations
    "wastage_by_city.csv": """
        SELECT location AS city,
               SUM(CASE WHEN status='Expired' THEN 1 ELSE 0 END) AS expired_items,
               SUM(CASE WHEN status='Available' THEN 1 ELSE 0 END) AS still_unclaimed,
               COUNT(*) AS total_items
        FROM food_listings
        GROUP BY location
        ORDER BY expired_items DESC, total_items DESC;
    """,
    # Expiry window (next 7 days)
    "soon_to_expire_7_days.csv": """
        SELECT food_id, food_name, quantity, expiry_date, location, status
        FROM food_listings
        WHERE expiry_date BETWEEN CURDATE() AND DATE_ADD(CURDATE(), INTERVAL 7 DAY)
        ORDER BY expiry_date ASC;
    """,
    # Demand vs Supply by city (claims vs available)
    "city_supply_demand.csv": """
        SELECT city,
               COALESCE(available,0) AS available_listings,
               COALESCE(claims,0) AS total_claims
        FROM
        (SELECT location AS city, COUNT(*) AS available
         FROM food_listings WHERE status IN ('Available','Reserved')
         GROUP BY location) a
        LEFT JOIN
        (SELECT fl.location AS city, COUNT(c.claim_id) AS claims
         FROM claims c JOIN food_listings fl ON fl.food_id=c.food_id
         GROUP BY fl.location) b
        USING (city)
        ORDER BY total_claims DESC, available_listings DESC;
    """,
    # Monthly trend of expired vs completed
    "monthly_trend_expired_completed.csv": """
        SELECT DATE_FORMAT(COALESCE(fl.expiry_date, c.timestamp), '%Y-%m') AS ym,
               SUM(CASE WHEN fl.status='Expired' THEN 1 ELSE 0 END) AS expired_listings,
               SUM(CASE WHEN c.status='Completed' THEN 1 ELSE 0 END) AS completed_claims
        FROM food_listings fl
        LEFT JOIN claims c ON c.food_id = fl.food_id
        GROUP BY ym
        ORDER BY ym;
    """
}

generated = []
for fname, q in reports.items():
    df = sql_df(q)
    out = f"/mnt/data/reports/{fname}"
    df.to_csv(out, index=False)
    generated.append(out)

generated


['/mnt/data/reports/wastage_by_food_type.csv',
 '/mnt/data/reports/wastage_by_city.csv',
 '/mnt/data/reports/soon_to_expire_7_days.csv',
 '/mnt/data/reports/city_supply_demand.csv',
 '/mnt/data/reports/monthly_trend_expired_completed.csv']

In [11]:
#writing streamlit application file app.py

%%writefile app.py
import streamlit as st
import pandas as pd
import mysql.connector as mysql
import plotly.express as px
from urllib.parse import quote
from datetime import datetime, date

DB_CONFIG = dict(
    host="127.0.0.1",
    user="root",
    password="root",
    database="foodwaste_db",
    auth_plugin="mysql_native_password"
)

def get_conn():
    return mysql.connect(**DB_CONFIG)

def sql_df(query, params=None):
    con = get_conn()
    df = pd.read_sql(query, con, params=params)
    con.close()
    return df

def sql_exec(query, params=None):
    con = get_conn()
    cur = con.cursor()
    cur.execute(query, params or ())
    con.commit()
    n = cur.rowcount
    cur.close(); con.close()
    return n

st.set_page_config(page_title="Food Wastage Management", layout="wide")
st.title("🍲 Local Food Wastage Management System")

# ---------------- Sidebar filters ----------------
with st.sidebar:
    st.header("🔎 Filters")
    cities = ["All"] + sql_df("SELECT DISTINCT location FROM food_listings ORDER BY location")["location"].dropna().tolist()
    providers = ["All"] + sql_df("SELECT DISTINCT name FROM providers ORDER BY name")["name"].dropna().tolist()
    foodtypes = ["All"] + sql_df("SELECT DISTINCT food_type FROM food_listings ORDER BY food_type")["food_type"].dropna().tolist()
    mealtypes = ["All"] + sql_df("SELECT DISTINCT meal_type FROM food_listings ORDER BY meal_type")["meal_type"].dropna().tolist()

    flt_city = st.selectbox("City", cities)
    flt_provider = st.selectbox("Provider", providers)
    flt_foodtype = st.selectbox("Food Type", foodtypes)
    flt_mealtype = st.selectbox("Meal Type", mealtypes)

tabs = st.tabs([
    "🏠 Browse & Contact",
    "🛠️ CRUD",
    "📑 Analytics (15 queries)",
    "📈 Data Analysis & Insights",
])

# ---------------- Tab 1: Browse & Contact ----------------
with tabs[0]:
    st.subheader("Available Donations")
    q = """
      SELECT fl.food_id, fl.food_name, fl.food_type, fl.meal_type, fl.quantity,
             fl.expiry_date, fl.location, fl.status,
             p.provider_id, p.name AS provider_name, p.contact AS provider_contact
      FROM food_listings fl
      JOIN providers p ON p.provider_id = fl.provider_id
      WHERE 1=1
    """
    params = []
    if flt_city != "All":
        q += " AND fl.location=%s"; params.append(flt_city)
    if flt_provider != "All":
        q += " AND p.name=%s"; params.append(flt_provider)
    if flt_foodtype != "All":
        q += " AND fl.food_type=%s"; params.append(flt_foodtype)
    if flt_mealtype != "All":
        q += " AND fl.meal_type=%s"; params.append(flt_mealtype)
    q += " ORDER BY fl.expiry_date ASC"

    df = sql_df(q, params or None)

    def contact_link(v):
        if not v or pd.isna(v): return ""
        v = str(v).strip()
        if "@" in v:
            return f"[Email](mailto:{v})"
        digits = "".join(ch for ch in v if ch.isdigit() or ch=="+")
        if digits:
            return f"[Call](tel:{digits}) • [WhatsApp](https://wa.me/{digits.lstrip('+')})"
        return f"[Contact](mailto:?subject={quote('Food Donation')}&body={quote(v)})"

    if not df.empty:
        df["Contact Provider"] = [contact_link(row["provider_contact"]) for _, row in df.iterrows()]
    st.dataframe(df, use_container_width=True)

# ---------------- Tab 2: CRUD ----------------
with tabs[1]:
    st.subheader("Create / Update / Delete records")

    crud_tabs = st.tabs(["Providers", "Receivers", "Food Listings", "Claims"])

    # Providers
    with crud_tabs[0]:
        st.markdown("### Providers")
        with st.form("prov_form"):
            pid = st.number_input("Provider ID", min_value=1, step=1)
            name = st.text_input("Name")
            ptype = st.text_input("Provider Type")
            contact = st.text_input("Contact (email/phone)")
            address = st.text_input("Address")
            city = st.text_input("City")
            submitted = st.form_submit_button("Save (Upsert)")
        if submitted:
            sql_exec("""
                INSERT INTO providers(provider_id,name,provider_type,contact,address,city)
                VALUES(%s,%s,%s,%s,%s,%s)
                ON DUPLICATE KEY UPDATE
                  name=VALUES(name), provider_type=VALUES(provider_type),
                  contact=VALUES(contact), address=VALUES(address), city=VALUES(city)
            """, (pid, name, ptype, contact, address, city))
            st.success("✅ Saved (inserted/updated).")

        # Delete
        pdata = sql_df("SELECT provider_id,name FROM providers ORDER BY provider_id")
        if not pdata.empty:
            del_pid = st.selectbox("Delete provider", pdata["provider_id"])
            if st.button("Delete Provider"):
                used = sql_df("SELECT COUNT(*) n FROM food_listings WHERE provider_id=%s", (del_pid,))["n"][0]
                if used:
                    st.error("Cannot delete: provider has listings.")
                else:
                    sql_exec("DELETE FROM providers WHERE provider_id=%s", (del_pid,))
                    st.success("Deleted.")

    # Receivers
    with crud_tabs[1]:
        st.markdown("### Receivers")
        with st.form("recv_form"):
            rid = st.number_input("Receiver ID", min_value=1, step=1)
            name = st.text_input("Name", key="rname")
            rtype = st.text_input("Type")
            contact = st.text_input("Contact")
            city = st.text_input("City", key="rcity")
            submitted = st.form_submit_button("Save (Upsert)")
        if submitted:
            sql_exec("""
                INSERT INTO receivers(receiver_id,name,type,contact,city)
                VALUES(%s,%s,%s,%s,%s)
                ON DUPLICATE KEY UPDATE
                  name=VALUES(name), type=VALUES(type),
                  contact=VALUES(contact), city=VALUES(city)
            """, (rid, name, rtype, contact, city))
            st.success("✅ Saved (inserted/updated).")

        rdata = sql_df("SELECT receiver_id,name FROM receivers ORDER BY receiver_id")
        if not rdata.empty:
            del_rid = st.selectbox("Delete receiver", rdata["receiver_id"])
            if st.button("Delete Receiver"):
                used = sql_df("SELECT COUNT(*) n FROM claims WHERE receiver_id=%s", (del_rid,))["n"][0]
                if used:
                    st.error("Cannot delete: receiver has claims.")
                else:
                    sql_exec("DELETE FROM receivers WHERE receiver_id=%s", (del_rid,))
                    st.success("Deleted.")

    # Food Listings
    with crud_tabs[2]:
        st.markdown("### Food Listings")
        with st.form("food_form"):
            fid = st.number_input("Food ID", min_value=1, step=1)
            pid = st.number_input("Provider ID (existing)", min_value=1, step=1, key="pfid")
            fname = st.text_input("Food Name")
            ftype = st.text_input("Food Type")
            mtype = st.text_input("Meal Type")
            qty = st.number_input("Quantity", min_value=0, step=1)
            exp = st.date_input("Expiry Date")
            loc = st.text_input("Location")
            status = st.selectbox("Status", ["Available","Reserved","Claimed","Expired"])
            ptype = st.text_input("Provider Type (optional)", key="ptype2")
            submitted = st.form_submit_button("Save (Upsert)")
        if submitted:
            sql_exec("""
                INSERT INTO food_listings
                (food_id,provider_id,food_name,food_type,meal_type,quantity,expiry_date,location,status,provider_type)
                VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                ON DUPLICATE KEY UPDATE
                  provider_id=VALUES(provider_id), food_name=VALUES(food_name),
                  food_type=VALUES(food_type), meal_type=VALUES(meal_type),
                  quantity=VALUES(quantity), expiry_date=VALUES(expiry_date),
                  location=VALUES(location), status=VALUES(status), provider_type=VALUES(provider_type)
            """, (fid, pid, fname, ftype, mtype, int(qty), str(exp), loc, status, ptype))
            st.success("✅ Saved (inserted/updated).")

        fdata = sql_df("SELECT food_id,food_name FROM food_listings ORDER BY food_id")
        if not fdata.empty:
            del_fid = st.selectbox("Delete listing", fdata["food_id"])
            if st.button("Delete Listing"):
                used = sql_df("SELECT COUNT(*) n FROM claims WHERE food_id=%s", (del_fid,))["n"][0]
                if used:
                    st.error("Cannot delete: listing has claims.")
                else:
                    sql_exec("DELETE FROM food_listings WHERE food_id=%s", (del_fid,))
                    st.success("Deleted.")

    # Claims
    with crud_tabs[3]:
        st.markdown("### Claims")
        with st.form("claim_form"):
            cid = st.number_input("Claim ID", min_value=1, step=1)
            fid = st.number_input("Food ID (existing)", min_value=1, step=1)
            rid = st.number_input("Receiver ID (existing)", min_value=1, step=1, key="rid2")
            status = st.selectbox("Status", ["Pending","Completed","Cancelled"])
            d = st.date_input("Date", value=date.today())
            t = st.time_input("Time", value=datetime.now().time())
            ts = datetime.combine(d, t)
            submitted = st.form_submit_button("Save (Upsert)")
        if submitted:
            sql_exec("""
                INSERT INTO claims(claim_id,food_id,receiver_id,status,timestamp)
                VALUES(%s,%s,%s,%s,%s)
                ON DUPLICATE KEY UPDATE
                  food_id=VALUES(food_id), receiver_id=VALUES(receiver_id),
                  status=VALUES(status), timestamp=VALUES(timestamp)
            """, (cid, fid, rid, status, ts))
            st.success("✅ Saved (inserted/updated).")

        cdata = sql_df("SELECT claim_id FROM claims ORDER BY claim_id")
        if not cdata.empty:
            del_cid = st.selectbox("Delete claim", cdata["claim_id"])
            if st.button("Delete Claim"):
                sql_exec("DELETE FROM claims WHERE claim_id=%s", (del_cid,))
                st.success("Deleted.")

# ---------------- Tab 3: Analytics (15 queries) ----------------
with tabs[2]:
    st.subheader("All 15 SQL Queries with outputs")

    queries = {
        "1) Providers & Receivers by City": """
            SELECT city,
                   COUNT(DISTINCT p.provider_id) AS providers,
                   (SELECT COUNT(DISTINCT r.receiver_id) FROM receivers r WHERE r.city=p.city) AS receivers
            FROM providers p GROUP BY city ORDER BY city;
        """,
        "2) Top Provider Type by Quantity": """
            SELECT provider_type, SUM(quantity) AS total_quantity
            FROM food_listings GROUP BY provider_type ORDER BY total_quantity DESC;
        """,
        "3) Receivers with Most Completed Claims": """
            SELECT r.receiver_id, r.name, COUNT(*) AS completed_claims
            FROM claims c JOIN receivers r ON r.receiver_id=c.receiver_id
            WHERE c.status='Completed'
            GROUP BY r.receiver_id, r.name ORDER BY completed_claims DESC;
        """,
        "4) Total Quantity Available": """
            SELECT SUM(quantity) AS total_available_quantity
            FROM food_listings WHERE status IN ('Available','Reserved');
        """,
        "5) City with Highest Listings": """
            SELECT location AS city, COUNT(*) AS listings
            FROM food_listings GROUP BY location ORDER BY listings DESC;
        """,
        "6) Most Common Food Types": """
            SELECT food_type, COUNT(*) AS n
            FROM food_listings GROUP BY food_type ORDER BY n DESC;
        """,
        "7) Claims per Food Item": """
            SELECT fl.food_id, fl.food_name, COUNT(c.claim_id) AS claims
            FROM food_listings fl LEFT JOIN claims c ON c.food_id=fl.food_id
            GROUP BY fl.food_id, fl.food_name ORDER BY claims DESC, fl.food_id;
        """,
        "8) Top Provider by Completed Claims": """
            SELECT p.provider_id, p.name, COUNT(*) AS completed_claims
            FROM claims c JOIN food_listings fl ON fl.food_id=c.food_id
            JOIN providers p ON p.provider_id=fl.provider_id
            WHERE c.status='Completed'
            GROUP BY p.provider_id, p.name ORDER BY completed_claims DESC;
        """,
        "9) Claim Status Percentages": """
            SELECT status, COUNT(*) AS cnt,
                   ROUND(100*COUNT(*)/(SELECT COUNT(*) FROM claims),2) AS pct
            FROM claims GROUP BY status;
        """,
        "10) Avg Quantity Claimed per Receiver": """
            SELECT r.receiver_id, r.name, AVG(fl.quantity) AS avg_claim_qty
            FROM claims c JOIN food_listings fl ON fl.food_id=c.food_id
            JOIN receivers r ON r.receiver_id=c.receiver_id
            WHERE c.status='Completed'
            GROUP BY r.receiver_id, r.name ORDER BY avg_claim_qty DESC;
        """,
        "11) Most Claimed Meal Type": """
            SELECT fl.meal_type, COUNT(*) AS completed_claims
            FROM claims c JOIN food_listings fl ON fl.food_id=c.food_id
            WHERE c.status='Completed'
            GROUP BY fl.meal_type ORDER BY completed_claims DESC;
        """,
        "12) Total Quantity by Provider": """
            SELECT p.provider_id, p.name, SUM(fl.quantity) AS total_qty
            FROM food_listings fl JOIN providers p ON p.provider_id=fl.provider_id
            GROUP BY p.provider_id, p.name ORDER BY total_qty DESC;
        """,
        "13) Soon-to-Expire Items (3 days)": """
            SELECT food_id, food_name, quantity, expiry_date, location
            FROM food_listings
            WHERE expiry_date BETWEEN CURDATE() AND DATE_ADD(CURDATE(), INTERVAL 3 DAY)
            ORDER BY expiry_date ASC;
        """,
        "14) Unclaimed by City": """
            SELECT location AS city, COUNT(*) AS unclaimed
            FROM food_listings WHERE status='Available'
            GROUP BY location ORDER BY unclaimed DESC;
        """,
        "15) Provider Efficiency (%)": """
            SELECT p.provider_id, p.name,
                   SUM(CASE WHEN c.status='Completed' THEN 1 ELSE 0 END) AS completed,
                   SUM(CASE WHEN c.status IN ('Completed','Cancelled','Pending') THEN 1 ELSE 0 END) AS attempts,
                   ROUND(100*SUM(CASE WHEN c.status='Completed' THEN 1 ELSE 0 END)/
                         NULLIF(SUM(CASE WHEN c.status IN ('Completed','Cancelled','Pending') THEN 1 ELSE 0 END),0),2) AS completion_rate_pct
            FROM providers p
            LEFT JOIN food_listings fl ON fl.provider_id=p.provider_id
            LEFT JOIN claims c ON c.food_id=fl.food_id
            GROUP BY p.provider_id, p.name
            ORDER BY completion_rate_pct DESC;
        """,
    }

    for title, q in queries.items():
        with st.expander(title, expanded=False):
            dfx = sql_df(q)
            st.dataframe(dfx, use_container_width=True)
            st.download_button("⬇️ CSV", dfx.to_csv(index=False).encode("utf-8"),
                               file_name=f"{title.replace(' ','_').lower()}.csv", mime="text/csv")

# ---------------- Tab 4: Data Analysis & Insights ----------------
with tabs[3]:
    st.subheader("Trends & Insights for Effective Distribution")

    # Trend 1: Top providers
    top_prov = sql_df("""
        SELECT p.name, COUNT(fl.food_id) AS listings, SUM(fl.quantity) AS total_qty
        FROM providers p LEFT JOIN food_listings fl ON fl.provider_id=p.provider_id
        GROUP BY p.name ORDER BY listings DESC, total_qty DESC LIMIT 10;
    """)
    if not top_prov.empty:
        st.markdown("**Top Providers by Listings**")
        st.plotly_chart(px.bar(top_prov, x="name", y="listings", title="Top Providers"), use_container_width=True)
        st.dataframe(top_prov, use_container_width=True)

    # Trend 2: Highest demand locations
    demand = sql_df("""
        SELECT fl.location, COUNT(c.claim_id) AS claims
        FROM claims c JOIN food_listings fl ON fl.food_id=c.food_id
        GROUP BY fl.location ORDER BY claims DESC;
    """)
    if not demand.empty:
        st.markdown("**Highest Demand Locations**")
        st.plotly_chart(px.bar(demand, x="location", y="claims", title="Claims by Location"), use_container_width=True)
        st.dataframe(demand, use_container_width=True)

    # Trend 3: Wastage (expired vs available) over time
    wastage = sql_df("""
        SELECT DATE(fl.expiry_date) AS day,
               SUM(CASE WHEN fl.status='Expired' THEN 1 ELSE 0 END) AS expired_items,
               SUM(CASE WHEN fl.status='Available' THEN 1 ELSE 0 END) AS still_unclaimed
        FROM food_listings fl
        GROUP BY DATE(fl.expiry_date) ORDER BY day;
    """)
    if not wastage.empty:
        st.markdown("**Wastage Trends (Expired vs Unclaimed)**")
        st.plotly_chart(px.line(wastage, x="day", y=["expired_items","still_unclaimed"], title="Daily Wastage Trend"),
                        use_container_width=True)
        st.dataframe(wastage, use_container_width=True)

    st.info("Download additional auto-generated reports from `/mnt/data/reports/` (created in the notebook).")


Overwriting app.py


In [ ]:
import os
from pyngrok import ngrok

# Kill any running tunnels
ngrok.kill()

# placeyour own ngrok auth token
os.environ["NGROK_AUTH_TOKEN"] = "myauthtoken"
ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])

# Create public URL
public_url = ngrok.connect(8501)
print(" Streamlit app available at:", public_url)

# Launch Streamlit
!streamlit run app.py --server.port 8501 --server.headless true

 Streamlit app available at: NgrokTunnel: "https://e86abd4a7aa0.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.58.234.20:8501



## The end
